In [2]:
import pandas as pd

In [3]:
# clean benchmark log
import csv

with open('benchmark_log.csv', 'r') as fin, open('benchmark_log_cleaned.csv', 'w') as fout:
    data = [row[:9] for row in csv.reader(fin)]
    csv.writer(fout).writerows(data)

In [4]:
data = pd.read_csv('benchmark_log_cleaned.csv')

# remove unnecessary columns
data.drop("train_steps", axis=1, inplace=True)

# get total cost in scientific notation
data.drop("total_cost_format", axis=1, inplace=True)
data["total_cost_format"] = data["total_cost"].apply(lambda x: f"{x:.4e}")

# convert microgrid to IDs
unique_microgrids = data["microgrid"].unique()
microgrid_map = {name: i for i, name in enumerate(unique_microgrids)}
data["microgrid"] = data["microgrid"].map(microgrid_map)

data.head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format
0,0,53062.168702,0,SB3Agent DQN,ReLU,"[64, 64]",0.0001,5.3062e+04
1,1,55578.578584,0,SB3Agent DQN,ReLU,"[64, 64]",0.0005,5.5579e+04
2,2,57543.274309,0,SB3Agent DQN,ReLU,"[64, 64]",0.0010,5.7543e+04
3,3,55934.748791,0,SB3Agent DQN,ReLU,"[128, 128]",0.0001,5.5935e+04
4,4,84487.924273,0,SB3Agent DQN,ReLU,"[128, 128]",0.0005,8.4488e+04


In [5]:
# baseline
data[data["agent"] == "BasicAgent heuristics"].drop_duplicates(subset=["microgrid"])[["total_cost"]]

,total_cost
36,1.871835e+05
75,2.369357e+07
114,3.600570e+07
153,3.593190e+07
192,3.244453e+07
231,1.135082e+08
270,4.628540e+07
309,1.544690e+07
348,9.735533e+06
387,3.571063e+06


In [6]:
heuristic_res = [
    item["total_cost"] for item in data[data["agent"] == "BasicAgent heuristics"]\
    .drop_duplicates(subset=["microgrid"])[["total_cost"]]\
    .to_dict(orient="records")
]

data = data[data["agent"] != "BasicAgent heuristics"].copy()

def add_baseline(row):
    return heuristic_res[row["microgrid"]]

def absolute_improvement(row):
    return heuristic_res[row["microgrid"]] - row["total_cost"]

def add_percentage_improvement(row):
    heuristic_cost = heuristic_res[row["microgrid"]]
    return (heuristic_cost - row["total_cost"]) / heuristic_cost * 100

data["baseline"] = data.apply(add_baseline, axis=1)
data["absolute_improvement"] = data.apply(absolute_improvement, axis=1)
data["percentage_improvement"] = data.apply(add_percentage_improvement, axis=1)

data.head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format,baseline,absolute_improvement,percentage_improvement
0,0,53062.168702,0,SB3Agent DQN,ReLU,"[64, 64]",0.0001,5.3062e+04,187183.490996,134121.322294,71.652324
1,1,55578.578584,0,SB3Agent DQN,ReLU,"[64, 64]",0.0005,5.5579e+04,187183.490996,131604.912412,70.307970
2,2,57543.274309,0,SB3Agent DQN,ReLU,"[64, 64]",0.0010,5.7543e+04,187183.490996,129640.216687,69.258360
3,3,55934.748791,0,SB3Agent DQN,ReLU,"[128, 128]",0.0001,5.5935e+04,187183.490996,131248.742206,70.117691
4,4,84487.924273,0,SB3Agent DQN,ReLU,"[128, 128]",0.0005,8.4488e+04,187183.490996,102695.566723,54.863581


In [7]:
[f"{res:.4e}" for res in heuristic_res]

['1.8718e+05',
 '2.3694e+07',
 '3.6006e+07',
 '3.5932e+07',
 '3.2445e+07',
 '1.1351e+08',
 '4.6285e+07',
 '1.5447e+07',
 '9.7355e+06',
 '3.5711e+06']

In [8]:
# get best general configs according to total_cost
data.sort_values("total_cost").head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format,baseline,absolute_improvement,percentage_improvement
34,34,51143.508873,0,SB3Agent PPO,Tanh,"[128, 128]",0.0005,5.1144e+04,187183.490996,136039.982124,72.67734
18,18,51143.508873,0,SB3Agent A2C,Tanh,"[64, 64]",0.0001,5.1144e+04,187183.490996,136039.982124,72.67734
17,17,51143.508873,0,SB3Agent A2C,ReLU,"[128, 128]",0.0010,5.1144e+04,187183.490996,136039.982124,72.67734
21,21,51143.508873,0,SB3Agent A2C,Tanh,"[128, 128]",0.0001,5.1144e+04,187183.490996,136039.982124,72.67734
15,15,51143.508873,0,SB3Agent A2C,ReLU,"[128, 128]",0.0001,5.1144e+04,187183.490996,136039.982124,72.67734


In [9]:
# get best general configs according to total_cost
data.sort_values("percentage_improvement", ascending=False).head()

,config_id,total_cost,microgrid,agent,policy_act,policy_net_arch,learning_rate,total_cost_format,baseline,absolute_improvement,percentage_improvement
202,202,4.073472e+06,5,SB3Agent DQN,Tanh,"[64, 64]",0.0005,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
230,230,4.073472e+06,5,SB3Agent PPO,Tanh,"[128, 128]",0.0010,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
229,229,4.073472e+06,5,SB3Agent PPO,Tanh,"[128, 128]",0.0005,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
227,227,4.073472e+06,5,SB3Agent PPO,Tanh,"[64, 64]",0.0010,4.0735e+06,1.135082e+08,1.094347e+08,96.411296
226,226,4.073472e+06,5,SB3Agent PPO,Tanh,"[64, 64]",0.0005,4.0735e+06,1.135082e+08,1.094347e+08,96.411296


In [10]:
# for each microgrid, get the best config
best_configs_lst = []
for microgrid in range(len(unique_microgrids)):
    best_config = data[data["microgrid"] == microgrid].sort_values("total_cost").head(1)
    best_configs_lst += best_config.to_dict(orient="records")

    print(f"Best config for microgrid {microgrid}:")
    print(best_config.to_string(index=False))
    print()

Best config for microgrid 0:
 config_id   total_cost  microgrid        agent policy_act policy_net_arch  learning_rate total_cost_format      baseline  absolute_improvement  percentage_improvement
        17 51143.508873          0 SB3Agent A2C       ReLU      [128, 128]          0.001        5.1144e+04 187183.490996         136039.982124                72.67734

Best config for microgrid 1:
 config_id   total_cost  microgrid        agent policy_act policy_net_arch  learning_rate total_cost_format     baseline  absolute_improvement  percentage_improvement
        39 3.646774e+06          1 SB3Agent DQN       ReLU        [64, 64]         0.0001        3.6468e+06 2.369357e+07          2.004679e+07               84.608591

Best config for microgrid 2:
 config_id   total_cost  microgrid        agent policy_act policy_net_arch  learning_rate total_cost_format     baseline  absolute_improvement  percentage_improvement
        78 1.750680e+06          2 SB3Agent DQN       ReLU        [64, 64]

In [11]:
res = pd.DataFrame(best_configs_lst).drop(["config_id", "total_cost_format"], axis=1)

res = res[["microgrid", "agent", "policy_act", "policy_net_arch", "learning_rate", "baseline", "total_cost", "absolute_improvement", "percentage_improvement"]]
res.columns = ["Microgrid", "Agent", "Policy Activation Function", "Policy Network Architecture", "Learning Rate", "Baseline Cost ($)", "Cost ($)", "Absolute Improvement ($)", "Relative Improvement (%)"]
res["Agent"] = res["Agent"].str.replace("SB3Agent", "")
res["Baseline Cost ($)"] = res["Baseline Cost ($)"].apply(lambda x: f"{x:.4e}")
res["Cost ($)"] = res["Cost ($)"].apply(lambda x: f"{x:.4e}")
res["Absolute Improvement ($)"] = res["Absolute Improvement ($)"].apply(lambda x: f"{x:.4e}")
res["Relative Improvement (%)"] = res["Relative Improvement (%)"].apply(lambda x: f"{x:.4f}")

res.to_latex("best_configs.tex", index=False, escape=False)
res

,Microgrid,Agent,Policy Activation Function,Policy Network Architecture,Learning Rate,Baseline Cost ($),Cost ($),Absolute Improvement ($),Relative Improvement (%)
0,0,A2C,ReLU,"[128, 128]",0.0010,1.8718e+05,5.1144e+04,1.3604e+05,72.6773
1,1,DQN,ReLU,"[64, 64]",0.0001,2.3694e+07,3.6468e+06,2.0047e+07,84.6086
2,2,DQN,ReLU,"[64, 64]",0.0001,3.6006e+07,1.7507e+06,3.4255e+07,95.1378
3,3,DQN,Tanh,"[64, 64]",0.0010,3.5932e+07,1.5799e+06,3.4352e+07,95.6031
4,4,DQN,ReLU,"[64, 64]",0.0001,3.2445e+07,4.3755e+06,2.8069e+07,86.5140
5,5,PPO,Tanh,"[128, 128]",0.0010,1.1351e+08,4.0735e+06,1.0943e+08,96.4113
6,6,PPO,Tanh,"[128, 128]",0.0010,4.6285e+07,1.7595e+06,4.4526e+07,96.1985
7,7,DQN,ReLU,"[64, 64]",0.0001,1.5447e+07,1.2147e+06,1.4232e+07,92.1360
8,8,PPO,Tanh,"[128, 128]",0.0010,9.7355e+06,3.6430e+06,6.0925e+06,62.5803
9,9,DQN,ReLU,"[64, 64]",0.0001,3.5711e+06,1.9401e+06,1.6310e+06,45.6723


In [15]:
res = data.drop(columns=["total_cost_format", "microgrid", "config_id", "baseline", "absolute_improvement", "percentage_improvement"])\
    .groupby(["agent", "policy_act", "policy_net_arch", "learning_rate"])\
    .mean()\
    .sort_values("total_cost")\
    .reset_index()\
    .rename(columns={
        "agent": "Agent",
        "policy_act": "Activation Function",
        "policy_net_arch": "Network Architecture",
        "learning_rate": "Learning Rate",
        "total_cost": "Average Cost ($)",
    })\
    .drop_duplicates(subset=["Average Cost ($)"])\

# res["% Improvement vs Next"] = (
#     (res["Average Cost ($)"].shift(-1) - res["Average Cost ($)"]) / res["Average Cost ($)"]
# ) * 100

# last_cost = res["Average Cost ($)"].iloc[-1]

# res["% Improvement vs Worst"] = (
#     (last_cost - res["Average Cost ($)"]) / last_cost
# ) * 100

res = res.head(10)

res["Agent"] = res["Agent"].str.replace("SB3Agent", "")
res

,Agent,Activation Function,Network Architecture,Learning Rate,Average Cost ($)
0,PPO,Tanh,"[64, 64]",0.0010,2.403480e+06
5,PPO,Tanh,"[64, 64]",0.0001,2.403504e+06
7,A2C,Tanh,"[64, 64]",0.0010,2.403517e+06
9,DQN,Tanh,"[128, 128]",0.0001,2.403525e+06
10,A2C,Tanh,"[128, 128]",0.0001,2.403624e+06
13,DQN,Tanh,"[64, 64]",0.0005,2.406824e+06
14,DQN,ReLU,"[64, 64]",0.0001,2.425341e+06
15,DQN,ReLU,"[128, 128]",0.0001,2.447507e+06
16,DQN,Tanh,"[64, 64]",0.0010,2.457281e+06
17,DQN,Tanh,"[128, 128]",0.0010,2.466014e+06


In [13]:
data.drop(columns=["total_cost_format", "microgrid", "config_id", "baseline", "absolute_improvement", "percentage_improvement", "learning_rate"])\
    .groupby(["agent", "policy_act", "policy_net_arch"])\
    .mean()\
    .sort_values("total_cost")

total_cost
agent        policy_act policy_net_arch              
SB3Agent PPO Tanh       [128, 128]       2.403488e+06
                        [64, 64]         2.403488e+06
SB3Agent A2C Tanh       [128, 128]       2.403540e+06
                        [64, 64]         2.403588e+06
SB3Agent DQN Tanh       [64, 64]         2.494487e+06
                        [128, 128]       2.507807e+06
             ReLU       [128, 128]       2.778062e+06
                        [64, 64]         2.994549e+06
SB3Agent PPO ReLU       [64, 64]         1.456596e+07
                        [128, 128]       4.063024e+07
SB3Agent A2C ReLU       [64, 64]         1.736087e+08
                        [128, 128]       3.041885e+08

In [14]:
res = data[data["microgrid"] == 5]\
    .drop(columns=["total_cost_format", "policy_act", "policy_net_arch", "microgrid", "config_id", "baseline", "absolute_improvement", "percentage_improvement", "learning_rate"])\
    .groupby(["agent"])\
    .agg({"total_cost": ["mean", "std"]})\
    .sort_values(("total_cost", "mean"))\
    .reset_index()

# combine mean and std to one column mean ± std
res["Cost ($)"] = res["total_cost"]["mean"].apply(lambda x: f"{x:.4e}") + " ± " + res["total_cost"]["std"].apply(lambda x: f"{x:.4e}")
res.drop(columns=["total_cost"])

/tmp/ipykernel_144731/67782488.py:10: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  res.drop(columns=["total_cost"])


,agent,Cost ($)
,,
0,SB3Agent DQN,4.5816e+06 ± 8.3395e+05
1,SB3Agent PPO,2.5073e+07 ± 2.5952e+07
2,SB3Agent A2C,3.5665e+08 ± 5.0252e+08


In [15]:
res = data.drop(columns=["total_cost_format", "microgrid", "config_id", "baseline", "total_cost", "absolute_improvement"])\
    .groupby(["agent", "policy_act", "policy_net_arch", "learning_rate"])\
    .agg(
        mean_percentage_improvement=("percentage_improvement", "mean"),
        std_percentage_improvement=("percentage_improvement", "std"),
    )\
    .sort_values("mean_percentage_improvement", ascending=False)\
    .reset_index()\
    .drop_duplicates(subset=["mean_percentage_improvement"])\
    .head(10)\
    .rename(columns={
        "agent": "Agent",
        "policy_act": "Activation Function",
        "policy_net_arch": "Network Architecture",
        "learning_rate": "Learning Rate",
    })

res["Agent"] = res["Agent"].str.replace("SB3Agent", "")
res["Relative Improvement (%)"] = res["mean_percentage_improvement"].apply(lambda x: f"{x:.4f}") + " ± " + res["std_percentage_improvement"].apply(lambda x: f"{x:.4f}")
res.drop(columns=["mean_percentage_improvement", "std_percentage_improvement"], inplace=True)

res

,Agent,Activation Function,Network Architecture,Learning Rate,Relative Improvement (%)
0,PPO,Tanh,"[64, 64]",0.0010,82.7539 ± 17.2398
5,PPO,Tanh,"[128, 128]",0.0001,82.7539 ± 17.2398
7,A2C,Tanh,"[128, 128]",0.0005,82.7538 ± 17.2397
9,A2C,Tanh,"[128, 128]",0.0001,82.7535 ± 17.2395
12,DQN,Tanh,"[128, 128]",0.0001,82.7299 ± 17.2556
13,DQN,ReLU,"[64, 64]",0.0001,82.6087 ± 17.2846
14,DQN,Tanh,"[64, 64]",0.0005,82.5263 ± 17.3915
15,DQN,Tanh,"[64, 64]",0.0010,82.4268 ± 17.3729
16,DQN,Tanh,"[128, 128]",0.0010,82.4264 ± 17.3922
17,DQN,ReLU,"[128, 128]",0.0001,82.3634 ± 17.4304
